# Tutorial 1: Spyker and PyTorch
In this tutorial we will see some usages of the library, and its interaction with PyTorch. We do this by creating a network to classify the MNIST dataset. First We start with importing the needed tools.

In [ ]:
import spyker, torch
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from torch.utils.data import TensorDataset, DataLoader

Then we load the MNIST dataset.

In [ ]:
batch, root = 64, './MNIST'
trainx, trainy, testx, testy = spyker.read_mnist(
    root+'/train-images-idx3-ubyte', root+'/train-labels-idx1-ubyte',
    root+'/t10k-images-idx3-ubyte', root+'/t10k-labels-idx1-ubyte')
trainx, trainy, testx, testy = spyker.to_torch(trainx, trainy, testx, testy)
train = DataLoader(TensorDataset(trainx, trainy), batch_size=batch)
test = DataLoader(TensorDataset(testx, testy), batch_size=batch)

We need to transform the input images into spikes. This is where the `Transform` module comes in. We define this module here.

In [ ]:
class Transform:
    def __init__(self, device):
        self.device = device
        self.filter = spyker.LoG(3, [.5, 1, 1.5, 2], pad=3, device=device, dtype='f32')
    
    def __call__(self, input):
        if self.device.kind == 'cuda': input = input.cuda()
        return spyker.code(spyker.threshold(self.filter(input), .01), 15)

Once we have our spikes, we need a network to process them. We define the `Network` module here.

In [ ]:
class Network:
    def __init__(self, device):
        self.conv1 = spyker.Conv(8, 32, 5, pad=2, device=device, dtype='f32')
        self.conv2 = spyker.Conv(32, 64, 3, pad=1, device=device, dtype='f32')
        self.conv1.stdpconfig = [spyker.STDPConfig(.004, -.003)]
        self.conv2.stdpconfig = [spyker.STDPConfig(.004, -.003)]
    
    def train1(self, input):
        output = spyker.inhibit(spyker.threshold(self.conv1(input), 16))
        self.conv1.stdp(input, spyker.convwta(output, 3, 5), spyker.fire(output))
        
    def train2(self, input):
        input = spyker.pool(spyker.fire(self.conv1(input), 16), 2)
        output = spyker.inhibit(spyker.threshold(self.conv2(input), 5))
        self.conv2.stdp(input, spyker.convwta(output, 1, 8), spyker.fire(output))
    
    def __call__(self, input):
        input = spyker.pool(spyker.fire(self.conv1(input), 16), 2)
        input = spyker.pool(spyker.fire(self.conv2(input), 5), 3)
        return (15 - spyker.gather(input)).flatten(1)

We need to get the output of the network for the entire datatset in order to use scikit-learn's tools on it. We define the `Total` function that does this operation here.

In [ ]:
def Total(network, transform, dataset):
    data_total, target_total = [], []
    for data, target in dataset:
        data_total.append(network(transform(data)).cpu())
        target_total.append(target)
    return torch.cat(data_total), torch.cat(target_total)

We need to update the learning rates of our STDP configurations after training on a specified number of samples. the `Update` function, which we define here, does this.

In [ ]:
def Update(config):
    rate = config.neg / config.pos
    pos = min(config.pos * 2, .1)
    config.pos, config.neg = pos, pos * rate

Spyker operations can be used on both CPUs and GPUs. Depending on the hardware of your machine and Spyker's build configuration, we need to specify the device to run our network on. We will use CUDA if it is avialable. If not, we fall back on the CPU instead.

In [ ]:
device = spyker.device('cuda' if spyker.cuda_available() else 'cpu')

We need instances of the `Transform` and `Network` modules. We create them here.

In [ ]:
transform = Transform(device)
network = Network(device)

Now that we have our instances, we train the layers of the network one by one.

In [ ]:
from tqdm.notebook import tqdm

for i, (data, _) in enumerate(tqdm(train, "Training Layer 1")):
    if (i + 1) % 10 == 0: Update(network.conv1.stdpconfig[0])
    network.train1(transform(data))
spyker.quantize(network.conv1.kernel, 0, .5, 1)

for i, (data, _) in enumerate(tqdm(train, "Training Layer 2")):
    if (i + 1) % 10 == 0: Update(network.conv2.stdpconfig[0])
    network.train2(transform(data))
spyker.quantize(network.conv2.kernel, 0, .5, 1);

Once we have trained our network, we get the output of the network for the training and testing sets.

In [ ]:
train_data, train_target = Total(network, transform, train)
test_data, test_target = Total(network, transform, test)

We can reduce the number of features of the network output to improve classification speed.

In [ ]:
pca = PCA(n_components=200).fit(train_data, train_target)
train_data, test_data = pca.transform(train_data), pca.transform(test_data)

Finally, we will run the SVM classifier and print the accuracy we get.

In [ ]:
target = SVC(C=2.4).fit(train_data, train_target).predict(test_data)
accuracy = (torch.tensor(target) == test_target).sum() / len(test_target)
print(f"Final Accuracy: {accuracy * 100 :.2f}%")

As we can see, our network reaches a good accuracy (if everything goes well, should be a little higher than 99%).